## Start Intepret Data

In [112]:
from pymongo import MongoClient, ASCENDING
client = MongoClient('localhost', 3001)

In [113]:
db = client.meteor
matches = db.matches
analytics = db.analytics

In [137]:
data = matches.find({'interpret_version': 0})

In [138]:
radiant_wins = 0
dire_wins = 0
first_match = None
latest_match = None 
hero_data = {}

for i in range(112):
    hero_data[i] = {}
    hero_data[i]["total_match"] = 0
    hero_data[i]["kills"] = 0
    hero_data[i]["deaths"] = 0
    hero_data[i]["assists"] = 0
    hero_data[i]["xp_per_min"] = 0
    hero_data[i]["gold_per_min"] = 0
    hero_data[i]["wins"] = 0
    hero_data[i]["loses"] = 0

for d in data:
    if d["game_mode"] in [1, 2, 3, 4, 5, 22]:
        if first_match == None:
            first_match = d["start_time"]
        
        if d["radiant_win"]:
            radiant_wins += 1
        else:
            dire_wins += 1
            
        # loop hero to get hero data
        for p in d["players"]:
            hero_data[p["hero_id"]]["total_match"] += 1
            hero_data[p["hero_id"]]["kills"] += p["kills"]
            hero_data[p["hero_id"]]["deaths"] += p["deaths"]
            hero_data[p["hero_id"]]["assists"] += p["assists"]
            hero_data[p["hero_id"]]["xp_per_min"] += p["xp_per_min"]
            hero_data[p["hero_id"]]["gold_per_min"] += p["gold_per_min"]
          
            if p < 10 and d["radiant_win"]:
                hero_data[p["hero_id"]]["wins"] += 1
            else:
                hero_data[p["hero_id"]]["loses"] += 1
                    
            if p > 10 and d["radiant_win"]:
                hero_data[p["hero_id"]]["loses"] += 1
            else:
                hero_data[p["hero_id"]]["wins"] += 1
            
        latest_match = d["start_time"]

    matches.update_one({"_id": d["_id"]}, {"$set": {"interpret_version": 1}})         
            

In [139]:
## Add to global data document
data = analytics.find_one({"type": "overall"})
if(data):
    analytics.update_one({"type": "overall"}, {"$set": {"radiant_wins": data["radiant_wins"] + radiant_wins, 
                                                       "dire_wins": data["dire_wins"] + dire_wins,
                                                        "latest_match": latest_match,
                                                       "total_matches": data["total_matches"] + radiant_wins + dire_wins}})
else:
    analytics.insert_one({"type": "overall", "first_match": first_match,
                            "latest_match": latest_match,
                          "radiant_wins": radiant_wins, "dire_wins": dire_wins, 
                          "total_matches": radiant_wins + dire_wins})

In [146]:
## Specific hero data

for key, value in hero_data.iteritems():
    if(value["total_match"] < 1):
        continue
    
    data = analytics.find_one({"type": "hero", "hero_id": key})
    if(data):
        avg_assists = float(value["assists"]) / float(value["total_match"])
        avg_deaths = float(value["deaths"]) / float(value["total_match"])
        avg_kills = float(value["kills"]) / float(value["total_match"])
        avg_gold_per_min = float(value["gold_per_min"]) / float(value["total_match"])
        avg_xp_per_min = float(value["xp_per_min"]) / float(value["total_match"])   
        avg_assists = (data["avg_assists"] + avg_assists) / 2.0
        avg_deaths = (data["avg_deaths"] + avg_deaths) / 2.0
        avg_kills = (data["avg_kills"] + avg_kills) / 2.0
        try:
            kd_ratio = avg_kills / avg_deaths
        except:
            kd_ratio = avg_kills
        avg_gold_per_min = (data["avg_gold_per_min"] + avg_gold_per_min) / 2.0
        avg_xp_per_min = (data["avg_xp_per_min"] + avg_xp_per_min) / 2.0
        
        twins = data["wins"] + value["wins"];
        wl_ratio = (float(data["wins"]) + float(value["wins"])) / (float(data["loses"]) + float(value["loses"]))
        
        analytics.update_one({"type": "hero", "hero_id": key}, {"$set": {
                              "avg_assists": avg_assists,
                              "avg_deaths": avg_deaths,
                              "avg_gold_per_min": avg_gold_per_min,
                              "avg_kills": avg_kills,
                              "loses": data["loses"] + value["loses"],
                              "wins": twins,
                              "kd_ratio": kd_ratio,
                              "wl_ratio": wl_ratio,
                              "avg_xp_per_min": avg_xp_per_min,
                              "total_match": data["total_match"] + value["total_match"]}})
        
    else:
        avg_assists = float(value["assists"]) / float(value["total_match"])
        avg_deaths = float(value["deaths"]) / float(value["total_match"])
        avg_kills = float(value["kills"]) / float(value["total_match"])
        try:
            kd_ratio = avg_kills / avg_deaths
        except:
            kd_ratio = avg_kills
        avg_gold_per_min = float(value["gold_per_min"]) / float(value["total_match"])
        avg_xp_per_min = float(value["xp_per_min"]) / float(value["total_match"])   
        
        analytics.insert_one({"type": "hero", 
                              "hero_id": key,
                              "avg_assists": avg_assists,
                              "avg_deaths": avg_deaths,
                              "avg_gold_per_min": avg_gold_per_min,
                              "avg_kills": avg_kills,
                              "loses": value["loses"],
                              "wins": value["wins"],
                              "kd_ratio": kd_ratio,
                              "wl_ratio": float(value["wins"]) / float(value["loses"]),
                              "avg_xp_per_min": avg_xp_per_min,
                              "total_match": value["total_match"]})

In [147]:
hero_data

{0: {'assists': 0,
  'deaths': 0,
  'gold_per_min': 0,
  'kills': 0,
  'loses': 0,
  'total_match': 0,
  'wins': 0,
  'xp_per_min': 0},
 1: {'assists': 0,
  'deaths': 0,
  'gold_per_min': 0,
  'kills': 0,
  'loses': 0,
  'total_match': 0,
  'wins': 0,
  'xp_per_min': 0},
 2: {'assists': 23,
  'deaths': 5,
  'gold_per_min': 612,
  'kills': 12,
  'loses': 2,
  'total_match': 1,
  'wins': 0,
  'xp_per_min': 743},
 3: {'assists': 0,
  'deaths': 0,
  'gold_per_min': 0,
  'kills': 0,
  'loses': 0,
  'total_match': 0,
  'wins': 0,
  'xp_per_min': 0},
 4: {'assists': 0,
  'deaths': 0,
  'gold_per_min': 0,
  'kills': 0,
  'loses': 0,
  'total_match': 0,
  'wins': 0,
  'xp_per_min': 0},
 5: {'assists': 0,
  'deaths': 0,
  'gold_per_min': 0,
  'kills': 0,
  'loses': 0,
  'total_match': 0,
  'wins': 0,
  'xp_per_min': 0},
 6: {'assists': 0,
  'deaths': 0,
  'gold_per_min': 0,
  'kills': 0,
  'loses': 0,
  'total_match': 0,
  'wins': 0,
  'xp_per_min': 0},
 7: {'assists': 10,
  'deaths': 6,
  'gold